# Titanic Solution with Pipeline 
- Build a model using Pipeline (make_pipeline)
- Use Column Transformers (make_column_transformer) to transform subsets of Variables
- Use (SimpleImputer) to treat Missing Values in both Numeric and Categorical Variables
- Use (KBinsDiscretizer, OneHotEncoder) for Binning and creating Dummy Variables
- Build our Model using Logistic Regression
- Measure Accuracy of the Model

## Import Libraries

In [1]:
import pandas as pd

import sklearn.model_selection as model_selection

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

#For Missing Value Treatment
from sklearn.impute import SimpleImputer

#For Binning and creating Dummy Variables
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder

from sklearn.linear_model import LogisticRegression

#Performance Check Libraries
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

C:\Users\DELL\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Import Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Check Information of Columns

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Divide Data into Target and X Variables

In [4]:
y = train['Survived']
X = train.drop(['Survived','PassengerId'], axis = 1)

## Split Titanic Train data into Train and Test Dataset

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2, random_state = 200)

## Classify Features based on how we want to deal with them

In [6]:
numeric_spl_features = ['Pclass','SibSp', 'Parch']

object_spl_features = ['Name', 'Ticket', 'Cabin']

numerical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind in ['i','f'] and c not in numeric_spl_features]
categorical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind not in ['i','f'] and c not in object_spl_features]

#cabin_spl_feature = ['Cabin']

#name_spl_feature = ['Name']

#ticket_spl_feature = ['Ticket']

In [7]:
print('Numerical : ' + str(numerical_features))
print('Categorical : ' + str(categorical_features))
print('Numeric Special : ' + str(numeric_spl_features))

Numerical : ['Age', 'Fare']
Categorical : ['Sex', 'Embarked']
Numeric Special : ['Pclass', 'SibSp', 'Parch']


## Make Column Transformer with multiple Pipeline

In [8]:
preprocessor = make_column_transformer(
    
    (make_pipeline(
    SimpleImputer(strategy = 'median'),
    KBinsDiscretizer(n_bins=4)), numerical_features),
    
    (make_pipeline(
    OneHotEncoder(categories = 'auto',handle_unknown = 'ignore')), numeric_spl_features),
    
    (make_pipeline(
    SimpleImputer(strategy = 'most_frequent'),
    OneHotEncoder(categories = 'auto', handle_unknown = 'ignore')), categorical_features),
    
)

## Make Logistic Model Pipeline

In [9]:
logModel = make_pipeline(preprocessor, LogisticRegression(solver = 'saga', max_iter = 1000))

## Fit Model 

In [13]:
logModel.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## Check Train and Test Scores

In [15]:
#Train_Score
train_scores = cross_val_score(logModel, X_train, y_train, cv = 7)

#Test Score
y_pred = logModel.predict(X_test).astype(int)
test_accuracy = accuracy_score(y_test, y_pred)

#Print Train and Test Score
print(f'Train Accuracy : {train_scores.mean():.3f}')
print(f'Test Accuracy : {test_accuracy:.3f}')

Train Accuracy : 0.810
Test Accuracy : 0.777


## Extract only X Variables for Prediction

In [13]:
X_submission = test.drop(['PassengerId'], axis = 1)
X_submission.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

## Make Prediction and Export CSV

In [14]:
prediction = logModel.predict(X_submission).astype(int)

In [ ]:
#Create Submission dataframe 
submission_df = pd.DataFrame({'PassengerId' : test['PassengerId'], 'Survived' : prediction})

submission_df.to_csv('LOG_Model_Pipeline.csv', index = False)

In [ ]:
## Scores 0.77033 when Submitted to Kaggle